In [1]:
import dill
import os
path = "./Manuales_migracion/"
manuales = []
for filename in os.listdir(path):
    with open(path + filename, 'rb') as file:
        manuales.append(dill.load(file))
# for m in manuales:
#     if m.title == "Manual Proceso de Migracion":
#         manuales.remove(m)
#         manual_general = m

    
def get_section(self, section_number):
    result = []
    for section_n in self.contents:
        if section_n.startswith(section_number):
            level = section_n.count('.') + 1
            result.append(level*"#" + self.index_map[section_n].strip() + ':\n' + self.contents[section_n])
    if len(result)==0:
        result = f"Section {section_number} doesnt exist"
    else:
        result =  "\n".join(result)
    return result

In [34]:
for instance in manuales:
    setattr(instance, 'get_section', get_section.__get__(instance, type(instance)))

In [7]:
manuales[3].get_section("1")

'#Ciclo de Migraciones:\n\n##Introducción:\nMANUAL DE INSTALADOR\n|\nDescripción del negocio\nLos objetivos de este documento son:\nDetallar el ciclo de migraciones.\nDescribir el diseño de las tablas correspondientes a bandejas de migraciones, enfocadas desde el punto de vista de Cuentas y Personas.\nComentar el detalle de los errores reportados por el proceso de migraciones.\nCiclo de Migraciones\nEn términos generales, la migración de Cuentas y Personas consiste en la generación de estructuras de “Clientes” que efectúan operaciones con el Banco y “No Clientes” (como por ejemplo proveedores, bancos corresponsales, empleados, etc.).  Asimismo, también se contempla la generación de las personas que integran dichas Cuentas.\nLa generación de estructuras se lleva a cabo en las siguientes instancias:\nGeneración de estructuras en Bandejas.\nControl de calidad de datos de Bandejas.\nPasaje de estructuras de tablas Bantotal.\nAdicionalmente, debe tenerse en cuenta que previo a la migración 

In [37]:
path = "./Manuales_migracion/"
for instance in manuales:
    print(path + instance.title)
    with open(path + instance.title + ".dill", 'wb') as file:
        dill.dump(file)

./Manuales_migracion/Manual Acuerdos de Sobregiro
./Manuales_migracion/Manual Acuerdos de Sobregiro
./Manuales_migracion/Manual Chequeras
./Manuales_migracion/Manual Chequeras
./Manuales_migracion/Manual Cuentas Vistas
./Manuales_migracion/Manual Cuentas Vistas
./Manuales_migracion/Manual Cuentas y Personas
./Manuales_migracion/Manual Cuentas y Personas
./Manuales_migracion/Manual Depositos
./Manuales_migracion/Manual Depositos
./Manuales_migracion/Manual Descuentos
./Manuales_migracion/Manual Descuentos
./Manuales_migracion/Manual Facultades
./Manuales_migracion/Manual Facultades
./Manuales_migracion/Manual Garantias
./Manuales_migracion/Manual Garantias
./Manuales_migracion/Manual Lineas de Credito
./Manuales_migracion/Manual Lineas de Credito
./Manuales_migracion/Manual Microfinanzas
./Manuales_migracion/Manual Microfinanzas
./Manuales_migracion/Manual Proceso de Migracion
./Manuales_migracion/Manual Proceso de Migracion
./Manuales_migracion/Manual Prestamos
./Manuales_migracion/Man

In [24]:
print(man_gar.get_index())

1 Ciclo de Migraciones
	1.1 Introducción
	1.2 Migraciones Previas
		1.2.1 Migración de Saldos Iniciales
		1.2.2 Migración de Clientes
	1.3 Generación de Estructuras de Bandejas
	1.4 Control de Calidad de Datos de Bandejas
		1.4.1 Programas de Control
		1.4.2 Estados de Control
		1.4.3 Salidas de Proceso
		1.4.4 Programas de Vuelco a Bantotal
	1.5 Secuencia de Ejecución
2 Diseño de Tablas de Bandejas
	2.1 Tabla BNJ002
3 Transacciones de Migraciones
	3.1 Migración Garantías Recibidas
		3.1.1 Productos Garantías Recibidas Predefinidos
		3.1.2 Transacción Migración Garantías Recibidas (módulo 499, transacción 210)
		3.1.3 Ajuste a la Operativa del Banco


In [54]:
import re


def parse_tables(input_string):
    # Dictionary to hold the result
    result = {}

    # Regular expression to capture table blocks and names
    table_blocks = re.findall(r'(\d+\.\d+ Tabla (\w+):.*?)(?=\d+\.\d+ Tabla |\Z)', input_string, re.DOTALL)

    for block, table_name in table_blocks:
        # Extract the description using a modified regular expression
        desc_match = re.search(r'\d+\.\d+ Tabla \w+:(.*?)(?=<table>)', block, re.DOTALL)
        description = desc_match.group(1).strip() if desc_match else "No description available"

        # Extract CSV records within <table> tags
        records_match = re.search(r'<table>\s*(.*?)\s*</table>', block, re.DOTALL)
        records = records_match.group(1) if records_match else "No records available"

        # Adding table info to the result dictionary
        result[table_name] = {
            "descripcion": description,
            "registros": records
        }

    return result


In [55]:
bandejas = {}
for manual in manuales:
    bandejas[manual.title] = parse_tables(manual.get_section("2"))

In [56]:
bandejas["Manual Cuentas y Personas"]["BJD002"]

{'descripcion': 'Almacena información relativa a todas las personas Físicas.\nTabla equivalente Bantotal: FSD002 y FSE002',
 'registros': 'NOMBRE,DESCRIPCIÓN,TIPO,DETALLE\nBD002Band,Bandeja,N (4.0),\nBD002Pais,País del Documento,N (3.0),País del documento de la persona (FST013).\nBD002Tdoc,Tipo de Documento,N (4.0),Tipo de documento de la persona (FST014).\nBD002Ndoc,Documento,C (25),Número de documento de la persona; no nulo. Dato Obligatorio.\nBD002Ape1,1er. Apellido,C (30),Primer Apellido.  Dato obligatorio.\nBD002Ape2,2o. Apellido,C (30),Segundo Apellido. Dato no obligatorio.\nBD002Nom1,1er. Nombre,C (25),Primer Nombre. Dato obligatorio.\nBD002Nom2,2do. Nombre,C (25),Segundo Nombre.  Dato no obligatorio.\nBD002Ebco,Empleado del Banco,C (1),Si es empleado del Banco ‘S’ o ‘N’.\nBD002Fibc,F/Ingreso al Banco,D,En caso de ser empleado; la fecha en que ingresó al Banco la cual debe ser menor al día de migración; si no es empleado; esta fecha debe ser nula.\nBD002Cant,Sexo,C (1),‘F’ femen

In [57]:
# guardo el diccionario en un archivo dill

with open('bandejas.dill', 'wb') as file:
    dill.dump(bandejas, file)


# Y lo levanto
with open('bandejas.dill', 'rb') as file:
    bandejas = dill.load(file)